In [1]:
# This is a very naive implementation and should just serve to understand the data
# We believe that a good solution will likely use deep-learning-based techniques

import cv2
from pathlib import Path

In [2]:
def find_location(query_image, ortho_image, method):
    best_score = None
    center = None

    # template matching for different scales
    scales = [0.05, 0.1, 0.2]

    for scale in scales:
        scaled_query_image = cv2.resize(
            query_image, None, fx=scale, fy=scale, interpolation=cv2.INTER_AREA
        )
        # template matching for different rotations
        for _ in range(4):
            res = cv2.matchTemplate(scaled_query_image, ortho_image, method)
            min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)

            if method in [cv2.TM_SQDIFF, cv2.TM_SQDIFF_NORMED]:
                top_left = min_loc
                current_score = -min_val
            else:
                top_left = max_loc
                current_score = max_val

            if not best_score or current_score > best_score:
                best_score = current_score
                center = (top_left[0] + scaled_query_image.shape[1]//2, top_left[1] + scaled_query_image.shape[0]//2)
                
            scaled_query_image = cv2.rotate(scaled_query_image, cv2.ROTATE_90_CLOCKWISE)

    return center, best_score

In [3]:
# DATA_PATH = Path("/kaggle/input/gnss-denied-localization/data")
DATA_PATH = Path("/home/rupert/projects/gnss-denied-localization/data")
TEST_IMAGE_PATH = DATA_PATH / "test_data" / "test_images"

ortho_image = cv2.imread(DATA_PATH / "map.png", cv2.IMREAD_GRAYSCALE)
image_paths = sorted(TEST_IMAGE_PATH.iterdir())

# This loop is very slow due to the naive template matching
print(f"{'id':>5s},{'x_pixel':>9s},{'y_pixel':>9s}")
for img_path in image_paths:
    img_id = int(img_path.stem)
    query_image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

    center, _ = find_location(query_image, ortho_image, cv2.TM_CCOEFF_NORMED)

    # print to terminal or write to .csv directly
    print(f"{img_id:5d},{center[0]:9d},{center[1]:9d}")

   id,  x_pixel,  y_pixel
    1,     3189,      323
    2,     4087,     1179
    3,     3241,      862
    4,     3626,     1626
    5,     3238,     1439
    6,     3245,     1719
    7,     2839,     1726
    8,     2914,     1597
    9,      640,     2191
   10,     3104,     2212
   11,     4509,      634
   12,     2788,     1361
   42,     3581,     1430
   43,     3156,      631
   44,     2891,      595
   53,     3173,     2174
   54,     3061,      914
   55,     3497,     2295
   56,      630,     2170
   65,     2745,     1607
   66,     3694,     1735
   67,     3233,     1853
   68,     4453,      540
 2468,     4494,     2000
 2469,     2269,     1471
 2470,     3638,     2065
 2471,     3700,     2067
 2472,     4075,     1144
 2473,     4349,      751
 2474,     4350,      686
 2475,     4348,      621
 2476,     3568,     1778
 2477,     4592,      559
 2478,     4534,      617
 2479,     4065,     1027
 2480,     3046,     2221
 2481,     4015,      949
 2482,     3